# compute cylindrical coordinates full 6D
(need to be adapted to read data from Gaia data archive)


In [1]:
from astropy.table import Table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mcolors
from astropy import units as u
from astropy.coordinates import SkyCoord
import sys
from os.path import expanduser
home = expanduser("~")
from pygaia.astrometry.vectorastrometry import astrometry_to_phase_space
import pygaia.astrometry.coordinates as coord
from pygaia.utils import degreesToRadians, radiansToDegrees

In [ ]:
def obs2cyl_astrometry(df0,ra_name='ra',dec_name='dec',pmra_name='pmra',pmdec_name='pmdec',dist_name='dist'
                   ,l_name='l',b_name='b',pml_name='pml',pmb_name='pmb'
                   ,Vl_name='Vl',Vb_name='Vb',X_name='X',Y_name='Y',Z_name='Z',R_name='R',phi_name='phi'
                   ,rs=8.178,Zs=0.0208,Us=11.1,Vs=248.5,Ws=7.25):
    k=4.7404705
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    
    
    
    
    rot3 = coord.CoordinateTransformation(coord.Transformations.ICRS2GAL)
    gal=rot3.transformSkyCoordinates(degreesToRadians(df0[ra_name]), degreesToRadians(df0[dec_name]))
    pmgal=rot3.transformProperMotions(degreesToRadians(df0[ra_name]),
                                      degreesToRadians(df0[dec_name]),df0[pmra_name],df0[pmdec_name])
    #ps=astrometryToPhaseSpace(gal[0], gal[1], 1./df0['dist'], pmgal[0], pmgal[1], pmgal[1]*0.)
    
    df2[l_name]=radiansToDegrees(gal[0])
    df2[b_name]=radiansToDegrees(gal[1])
    df2[pml_name]=pmgal[0]
    df2[pmb_name]=pmgal[1]
    print(pmgal[0])

    solarreflex_Vl=-Us*np.sin(gal[0])+Vs*np.cos(gal[0])
    solarreflex_Vb=(-Us*np.cos(gal[0])-Vs*np.sin(gal[0]))*np.sin(gal[1])+Ws*np.cos(gal[1])
  
    #print(len(df2[pml_name]),len(df0[dist_name]),len(solarreflex_Vl))
    df[Vl_name]=df2[pml_name]*k*df0[dist_name]+solarreflex_Vl
    df[Vb_name]=df2[pmb_name]*k*df0[dist_name]+solarreflex_Vb
    df[X_name]=(1.*df0[dist_name])*np.cos(df2[l_name]*np.pi/180.)*np.cos(df2[b_name]*np.pi/180.)
    df[Y_name]=(1.*df0[dist_name])*np.sin(df2[l_name]*np.pi/180.)*np.cos(df2[b_name]*np.pi/180.)
    df[Z_name]=(1.*df0[dist_name])*np.sin(df2[b_name]*np.pi/180.)
    
    #print('1',ps[4])
    #print('2',df2[pml_name]*k*df0[dist_name])

    
    dgc= rs    
    Xs=np.sqrt(dgc**2.-Zs**2.)
    costheta, sintheta= Xs/dgc, -Zs/dgc
    XYZ= np.dot(np.array([[costheta,0.,-sintheta], [0.,1.,0.],[sintheta,0.,costheta]]),
                  np.array([df[X_name]-dgc,df[Y_name],df[Z_name]])).T

    df[X_name]=XYZ[:,0]
    df[Y_name]=XYZ[:,1]
    df[Z_name]=XYZ[:,2]
    df[R_name]=np.sqrt(df[Y_name]**2.+df[X_name]**2.)
    df[phi_name]=np.degrees(np.arctan2(-XYZ[:,1],-XYZ[:,0]))

    
    return df,df2

def absG(df0,phot_g_mean_mag_name='phot_g_mean_mag',dist_name='dist',MG_name='MG'):
    df = pd.DataFrame()
    df[MG_name]=df0[phot_g_mean_mag_name]-5.*np.log10(1000.*df0[dist_name])+5.
    return df

#TRANSFORMATION BETWEEN DIFFERENT COORDINATE SYSTEMS
# these transformations are valid for right-handed systems only
# it means that phi goes counter-clockwise, and vphi is negative for standard disk rotation

#in all cases Vs=Vc+Vpec

##############################################
# general rotation of coordinate system by an angle phi
def rot(phi,X,Y):
    s=np.sin(phi)
    c=np.cos(phi)  
    X2=X*c-Y*s
    Y2=X*s+Y*c
    return X2,Y2
##############################################
# from cartesian to cylindric
def cart2cyl(X,Y,Z,VX,VY,VZ):
    phi=np.arctan2(-Y,-X)
    R=np.sqrt(X*X+Y*Y)
    VR,Vphi=rot(-(np.pi+phi),VX,VY)
    return R,phi,Z,VR,Vphi,VZ
# from cylindric to cartesian
def cyl2cart(R,phi,Z,VR,Vphi,VZ):
    X=-R*np.cos(phi)
    Y=-R*np.sin(phi)
    VX,VY=rot(np.pi+phi,VR,Vphi)
    return X,Y,Z,VX,VY,VZ
##############################################
#from cylindrical to solar (Vt, Vlos) 
def cyl2solar(R,phi,VR,Vphi,rs=8.178,Z=0.,VZ=0.,Us=11.1,Vs=248.5,Ws=0.):
    X=-R*np.cos(phi)
    Y=-R*np.sin(phi)
    lon=np.arctan2(Y,rs+X)
    vlos,vtl=rot(phi+np.pi-lon,VR,Vphi)
    c1,c2=add_solamotiontosolar(lon,0.,Us=Us,Vs=Vs,Ws=Ws)
    vlos=vlos-c1
    vtl=vtl-c2
    return vlos,vtl
##############################################
#add solar reflex motion to solar-reference system velocities to make them galactocentric
def add_solamotiontosolar(lon,lat,Us=11.1,Vs=248.5,Ws=7.25):    
    solarreflex_Vlos=(Us*np.cos(lon)+Vs*np.sin(lon))*np.cos(lat)+Ws*np.sin(lat)
    solarreflex_Vl=-Us*np.sin(lon)+Vs*np.cos(lon)
    solarreflex_Vb=(-Us*np.cos(lon)-Vs*np.sin(lon))*np.sin(lat)+Ws*np.cos(lat)
    return solarreflex_Vlos,solarreflex_Vl

def obs2cyl(df0,ra_name='ra',dec_name='dec',pmra_name='pmra',pmdec_name='pmdec',dist_name='dist'
                   ,l_name='l',b_name='b',pml_name='pml',pmb_name='pmb',radial_velocity_name='radial_velocity'
                   ,VR_name='VR',Vphi_name='Vphi',VZ_name='VZ',X_name='X',Y_name='Y',Z_name='Z'
                     ,R_name='R',phi_name='phi'
                   ,rs=8.178,Zs=0.0208,Us=11.1,Vs=248.5,Ws=7.25):
    
    
    df = pd.DataFrame()
    rot3 = coord.CoordinateTransformation(coord.Transformations.ICRS2GAL)
    gal=rot3.transformSkyCoordinates(degreesToRadians(df0[ra_name]), degreesToRadians(df0[dec_name]))
    pmgal=rot3.transformProperMotions(degreesToRadians(df0[ra_name]), degreesToRadians(df0[dec_name]),df0[pmra_name],df0[pmdec_name])
    
    ps=astrometryToPhaseSpace(gal[0], gal[1], 1./df0[dist_name], pmgal[0], pmgal[1], df0[radial_velocity_name])
    #ps=astrometryToPhaseSpace(gal[0], gal[1], 1./df0['dist'], pmgal[0], pmgal[1], pmgal[1]*0.)

    X=ps[0]/1000.
    Y=ps[1]/1000.
    Z=ps[2]/1000.

    #transform to put sun at -X axis WRONG!!
    #X=-rs+X
    #Y=Y
    #Z=Z+Zs

    dgc= rs    
    Xs=np.sqrt(dgc**2.-Zs**2.)
    costheta, sintheta= Xs/dgc, -Zs/dgc
    XYZ= np.dot(np.array([[costheta,0.,-sintheta], [0.,1.,0.],[sintheta,0.,costheta]]),
                  np.array([X-dgc,Y,Z])).T

    #print(ps[3])
    #print(ps[4])
    #print(ps[5])

    #add solar motion
    VX=ps[3]+Us
    VY=ps[4]+Vs
    VZ=ps[5]+Ws
    
    VXVYVZ= np.dot(np.array([[costheta,0.,-sintheta], [0.,1.,0.],[sintheta,0.,costheta]]),
                  np.array([VX,VY,VZ])).T
    
    #vsunin= np.dot(np.array([[costheta,0.,-sintheta], [0.,1.,0.],[sintheta,0.,costheta]]),
    #              np.array([Us,Vs,Ws])).T
    #print(vsunin)
    
    #print(' ')
    #print(VXVYVZ[:,0])
    #print(VXVYVZ[:,1])
    #print(VXVYVZ[:,2])    
    
    
    R,phi,Z,VR,Vphi,VZ=cart2cyl(XYZ[:,0],XYZ[:,1],XYZ[:,2],VXVYVZ[:,0],VXVYVZ[:,1],VXVYVZ[:,2])
    
    df[VR_name]=VR
    df[Vphi_name]=Vphi
    df[VZ_name]=VZ
    df[X_name]=XYZ[:,0]
    df[Y_name]=XYZ[:,1]
    df[Z_name]=XYZ[:,2]
    df[R_name]=R
    df[phi_name]=np.degrees(phi)

    #print(gal[0]*180./np.pi)
    #print(Vphi)
    
    
    #print(' ')
    #print(VR)
    #print(Vphi)
    #print(VZ) 

    return df



In [2]:
#reading main file

#6f edr3int4
path0=home+"/EDR3_anticenter/files/edr3int4/"
file0='edr3int4_RVS_full'

file1=path0+file0+'.h5'
df = pd.read_hdf(file1, "table")
print(df.columns.values.tolist())
print(len(df))

['dec', 'dr2_radial_velocity', 'dr2_radial_velocity_error', 'parallax', 'parallax_error', 'pmdec', 'pmdec_error', 'pmra', 'pmra_error', 'ra', 'source_id']
6156684


In [3]:
#establising distance used and name!!!!!!
file0='edr3int4_piepi3_zp17_RVS'

df['dist']=1./(df['parallax']-(-0.017)) #for the full 6d sample e.g. or four 1/pi

print(df.columns.values.tolist())

['dec', 'dr2_radial_velocity', 'dr2_radial_velocity_error', 'parallax', 'parallax_error', 'pmdec', 'pmdec_error', 'pmra', 'pmra_error', 'ra', 'source_id', 'dist']


In [4]:
df['radial_velocity']=df['dr2_radial_velocity']

# computing 6d phase space

In [5]:
ind=df['parallax']/df['parallax_error']>3
df=df[ind]

In [6]:
df_GAL3=obs2cyl(df)

In [37]:
df_GAL3['source_id']=df['source_id'].copy()

In [38]:
fileout=file0+'_GAL6D'
print(path0+fileout)
print(len(df_GAL3))
path0=home+"/EDR3_anticenter/files/edr3int4/"

/Users/tantoja/EDR3_anticenter/files/edr3int4/edr3int4_piepi3_zp17_RVS_GAL6D
6156684


In [39]:
#write in h5
df_GAL3.to_hdf(path0+fileout+'.h5', "table", append=False)
#write in fits
Table.from_pandas(df_GAL3).write(path0+fileout+'.fits', overwrite=True)
#write in csv
#df_GAL.to_csv(path_or_buf=path0+fileout+'.csv',index=False)